In [47]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [48]:
data_lg = pd.read_csv('epi_r.csv')

# round the ratings into a classification
data_lg['rating_rnd'] = data_lg['rating'].round()

#add a binary as well
data_lg['good_rating'] = np.where(data_lg['rating_rnd'] >= 4, 1, 0)

data_sm = data_lg.drop(['calories', 'protein', 'fat', 'sodium'], axis = 1)

In [49]:
# with nutritional data

data_lg = data_lg.dropna()

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectKBest, f_classif

X = data_lg.drop(['rating_rnd', 'rating', 'title', 'good_rating'], 1)
Y = data_lg.good_rating

selector=SelectKBest(score_func=f_classif,k=50)
selector.fit(X,Y)
scores = pd.DataFrame()
scores["Attribute"] = X.columns
scores["F Score"] = selector.scores_
scores["P Value"] = selector.pvalues_
scores["Support"] = selector.get_support()

scores = scores_sm.sort_values(by=['F Score'], ascending=False)

key_vals_lg = scores.Attribute.head(50).values

/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 97 199 225 229 247 292 298 323 385 401 528 647] are constant.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [50]:
# with nutritional data

data_sm = data_sm.dropna()

X = data_sm.drop(['rating_rnd', 'rating', 'title', 'good_rating'], 1)
Y = data_sm.good_rating

selector=SelectKBest(score_func=f_classif,k=50)
selector.fit(X,Y)
scores_sm = pd.DataFrame()
scores_sm["Attribute"] = X.columns
scores_sm["F Score"] = selector.scores_
scores_sm["P Value"] = selector.pvalues_
scores_sm["Support"] = selector.get_support()

scores_sm = scores_sm.sort_values(by=['F Score'], ascending=False)

key_vals_sm = scores_sm.Attribute.head(50).values

### Splitting the Test Set

In [59]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

X1 = data_sm.loc[:, key_vals_sm]
y1 = data_sm.good_rating

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = .2)

### First Model – Small Dataset Binary Classification

In [60]:
from sklearn.svm import SVC

svc_1 = SVC(kernel = 'linear')
svc_1.fit(X1_train, y1_train)

y1_pred_ = svc_1.predict(X1_test)

svc_1_cfmat = confusion_matrix(y1_test, y1_pred_, labels = [0, 1])
svc_1_cvscores = cross_val_score(svc_1, X1, y1, cv = 10)
svc_1_trainscore = svc_1.score(X1_train, y1_train)
svc_1_testscore = svc_1.score(X1_test, y1_test)

In [61]:
print("The coefficients are: \n {}".format(svc_1.coef_))

The coefficients are: 
 [[ 6.39070697e-05 -3.97928163e-04 -8.57139966e-01 -1.14297878e+00
   8.31390858e-05  2.66413604e-06 -1.00807275e-04 -3.34572618e-04
  -1.14234675e+00 -2.00006440e+00 -3.68005274e-04 -1.86038993e-04
  -1.37854493e-04  5.94657528e-05 -7.29079077e-05 -2.00032196e+00
  -1.38521891e-04 -1.03328718e-05 -3.09209657e-04 -8.75604485e-05
   4.50837152e-06 -2.71394565e-05  9.09115677e-05 -1.14290695e+00
  -7.51666915e-05 -2.00018903e+00 -5.49432252e-05 -5.41460190e-05
  -3.64122775e-04 -4.25258209e-05 -1.19960455e-04 -8.58062677e-05
  -8.57886413e-01  2.08445256e-04 -8.56391263e-01 -6.00210321e-05
  -2.10542642e-04 -9.71516669e-06 -3.08789383e-06 -8.16391940e-06
  -2.28672253e-04 -2.00000000e+00 -1.10990142e-04 -9.71155292e-06
  -2.00000000e+00 -2.80765613e-05 -5.56622776e-05 -8.56814834e-01
  -2.00000000e+00 -2.29651966e-04]]


In [62]:
print("The CV Scores are: \n {}".format(svc_1_cvscores))

print("The Mean and STD of the CV Scores are:\n Mean:{}. STD: {}.".format(svc_1_cvscores.mean(), svc_1_cvscores.std()))

The CV Scores are: 
 [0.79561316 0.80408774 0.79860419 0.80109671 0.80259222 0.80698254
 0.81047382 0.80788423 0.80489022 0.80738523]
The Mean and STD of the CV Scores are:
 Mean:0.8039610057687128. STD: 0.004335927448781084.


In [94]:
print("The Training score is: {}".format(svc_1_trainscore))
print("The Test score is: {}".format(svc_1_testscore))
print(svc_1_cfmat)

The Training score is: 0.025746524530889596
The Test score is: 0.027673896783844427
[[  88  717]
 [  39 3167]]


In [66]:
from sklearn.metrics import classification_report
print('Classification Report:\n {}'.format(classification_report(y1_test, y1_pred_)))

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.11      0.19       805
           1       0.82      0.99      0.89      3206

   micro avg       0.81      0.81      0.81      4011
   macro avg       0.75      0.55      0.54      4011
weighted avg       0.79      0.81      0.75      4011



##### Analysis
The small dataset performs pretty well on first glance of the scores. The r squared and cross validation scores all perform well and seem to show that there isn't any overfitting with the data. The more interesting aspects of the data are when you dive into the classification report. You can see that the data performs very very well when predicting if the review is a positive. When predicting a negative review it does a decent job of delivering the relevant results, but the recall is very worryingly low.

Let's see if adding the nutrition information back in helps.

___

### Second Model – Large Dataset Binary Classification

In [51]:
# Split the data
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

X2 = data_lg.loc[:, key_vals_lg]
y2 = data_lg.good_rating

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = .2)

In [52]:
from sklearn.svm import SVC

svc_2 = SVC(kernel = 'linear')
svc_2.fit(X2_train, y2_train)

y2_pred_ = svc_2.predict(X2_test)

svc_2_cfmat = confusion_matrix(y2_test, y2_pred_, labels = [0, 1])
svc_2_cvscores = cross_val_score(svc_2, X2, y2, cv = 10)
svc_2_trainscore = svc_2.score(X2_train, y2_train)
svc_2_testscore = svc_2.score(X2_test, y2_test)

In [53]:
print("The coefficients are: \n {}".format(svc_2.coef_))

The coefficients are: 
 [[-3.67742970e-04  2.78267416e-05 -2.80494329e-04 -1.99903000e+00
  -1.73126313e-04 -3.22752559e-04 -1.97700462e-05 -1.99853380e+00
   5.52708048e-05 -2.00003181e+00 -5.40642391e-05 -3.99752142e-05
  -1.99756496e-04  2.01104876e-04 -2.56384347e-05 -4.66709959e-06
  -2.78737436e-04  8.47390468e-05 -3.99198447e-04 -3.42542325e-06
   5.50941921e-07 -4.56791426e-05 -8.63715591e-04  1.73263789e-04
  -1.59570522e-04 -2.34830073e-04 -1.55991868e-05 -3.85834813e-05
   4.20574065e-06 -7.65334381e-05  0.00000000e+00  0.00000000e+00
  -1.26746392e-04 -3.76904177e-04 -5.60865814e-05  3.38601076e-04
   4.03576100e-05  1.17058400e-05 -9.68396257e-05 -1.99905637e+00
   4.53207109e-05  6.16425594e-05  3.87860051e-06  1.82301677e-05
  -6.43934455e-05 -2.00000000e+00 -1.63723671e-05 -1.99957512e+00
  -1.00000000e+00 -1.15343543e-05]]


In [54]:
print("The CV Scores are: \n {}".format(svc_2_cvscores))

print("The Mean and STD of the CV Scores are:\n Mean:{}. STD: {}.".format(svc_2_cvscores.mean(), svc_2_cvscores.std()))

The CV Scores are: 
 [0.81285444 0.81222432 0.8147448  0.81222432 0.81715006 0.81273644
 0.82030265 0.81651955 0.81336696 0.81904161]
The Mean and STD of the CV Scores are:
 Mean:0.8151165165265386. STD: 0.0028127667751303618.


In [95]:
print("The Training score is: {}".format(svc_2_trainscore))
print("The Test score is: {}".format(svc_2_testscore))
print(svc_2_cfmat)

The Training score is: 0.8143566306831613
The Test score is: 0.8172076898833911
[[  52  554]
 [  26 2541]]


In [67]:
from sklearn.metrics import classification_report
print('Classification Report:\n {}'.format(classification_report(y2_test, y2_pred_)))

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.09      0.15       606
           1       0.82      0.99      0.90      2567

   micro avg       0.82      0.82      0.82      3173
   macro avg       0.74      0.54      0.52      3173
weighted avg       0.79      0.82      0.76      3173



##### Analysis
The large dataset performs much like the small dataset, it similarly is able to predict a positive review with higher accuracy than it does a negative review. Funnily enough, when dealing with a binary classification it seems like adding in the nutrition info doesn't really help all that much the precision and recall are very similar for both. While the base score itself seems better for the large dataset, viewing the classification report shows a marginal difference between the two.

Let's see if turning this into a classification changes anything.

---

### 3rd Model – Small Dataset, 1–5 Classification

In [68]:
# with nutritional data

data_sm = data_sm.dropna()

X = data_sm.drop(['rating_rnd', 'rating', 'title', 'good_rating'], 1)
Y = data_sm.rating_rnd

selector=SelectKBest(score_func=f_classif,k=50)
selector.fit(X,Y)
scores_sm = pd.DataFrame()
scores_sm["Attribute"] = X.columns
scores_sm["F Score"] = selector.scores_
scores_sm["P Value"] = selector.pvalues_
scores_sm["Support"] = selector.get_support()

scores_sm = scores_sm.sort_values(by=['F Score'], ascending=False)

key_vals_sm2 = scores_sm.Attribute.head(50).values

In [80]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

X3 = data_sm.loc[:, key_vals_sm2]
y3 = data_sm.rating_rnd

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = .2)

In [81]:
from sklearn.svm import SVC

svc_3 = SVC(kernel = 'linear')
svc_3.fit(X3_train, y3_train)

y3_pred_ = svc_3.predict(X3_test)

svc_3_cfmat = confusion_matrix(y3_test, y3_pred_, labels = [0, 1])
svc_3_cvscores = cross_val_score(svc_3, X3, y3, cv = 10)
svc_3_trainscore = svc_3.score(X3_train, y3_train)
svc_3_testscore = svc_3.score(X3_test, y3_test)

In [82]:
print("The CV Scores are: \n {}".format(svc_3_cvscores))

print("The Mean and STD of the CV Scores are:\n Mean:{}. STD: {}.".format(svc_3_cvscores.mean(), svc_3_cvscores.std()))

The CV Scores are: 
 [0.66865969 0.6651719  0.67065272 0.66467364 0.67547358 0.6779661
 0.6751497  0.67365269 0.66949576 0.67716142]
The Mean and STD of the CV Scores are:
 Mean:0.6718057199007369. STD: 0.004540016547197042.


In [83]:
print("The Training score is: {}".format(svc_3_trainscore))
print("The Test score is: {}".format(svc_3_testscore))

The Training score is: 0.6746462190636494
The Test score is: 0.6646721515831463


In [84]:
from sklearn.metrics import classification_report
print('Classification Report:\n {}'.format(classification_report(y3_test, y3_pred_)))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.57      0.19      0.28       395
         1.0       0.00      0.00      0.00        35
         2.0       0.00      0.00      0.00       131
         3.0       0.00      0.00      0.00       286
         4.0       0.68      0.98      0.80      2623
         5.0       0.24      0.05      0.08       541

   micro avg       0.66      0.66      0.66      4011
   macro avg       0.25      0.20      0.19      4011
weighted avg       0.53      0.66      0.56      4011



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##### Analysis
When breaking the data out into a full classification, you begin to see a bit of why the model performed the way it did on the binary classifier. There is an overwhelming majority of 4 star reviews in the data and that seems to sway the results in it's direction. 

Similarly to the binary models, the preliminary numbers behind how the classifier performs look good but when you look at the classification report it shows a bit of a different story.

---

### 4th Model – Large Dataset, 1–5 Classification

In [85]:
# with nutritional data

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectKBest, f_classif

X = data_lg.drop(['rating_rnd', 'rating', 'title', 'good_rating'], 1)
Y = data_lg.rating_rnd

selector=SelectKBest(score_func=f_classif,k=50)
selector.fit(X,Y)
scores = pd.DataFrame()
scores["Attribute"] = X.columns
scores["F Score"] = selector.scores_
scores["P Value"] = selector.pvalues_
scores["Support"] = selector.get_support()

scores = scores_sm.sort_values(by=['F Score'], ascending=False)

key_vals_lg2 = scores.Attribute.head(50).values

/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 97 199 225 229 247 292 298 323 385 401 528 647] are constant.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [89]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

X4 = data_lg.loc[:, key_vals_lg2]
y4 = data_lg.rating_rnd

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size = .2)

In [90]:
from sklearn.svm import SVC

svc_4 = SVC(kernel = 'linear')
svc_4.fit(X4_train, y4_train)

y4_pred_ = svc_4.predict(X4_test)

svc_4_cfmat = confusion_matrix(y4_test, y4_pred_, labels = [0, 1, 2 , 3, 4, 5])
svc_4_cvscores = cross_val_score(svc_4, X4, y4, cv = 10)
svc_4_trainscore = svc_4.score(X4_train, y4_train)
svc_4_testscore = svc_4.score(X4_test, y4_test)

In [91]:
print("The CV Scores are: \n {}".format(svc_4_cvscores))

print("The Mean and STD of the CV Scores are:\n Mean:{}. STD: {}.".format(svc_4_cvscores.mean(),
                                                                          svc_4_cvscores.std()))

The CV Scores are: 
 [0.68344871 0.67589679 0.6809314  0.68010076 0.69143577 0.68241966
 0.68497475 0.68686869 0.68603917 0.69235629]
The Mean and STD of the CV Scores are:
 Mean:0.684447197339532. STD: 0.004791031114316742.


In [92]:
print("The Training score is: {}".format(svc_4_trainscore))
print("The Test score is: {}".format(svc_4_testscore))
print(svc_4_cfmat)
#print("Specificity:{}".format(svc_2_cfmat[0:1, 0:1]/(svc_2_cfmat[0:1, 0:1] + svc_2_cfmat[0:1, 1:2])))
#print("Sensitivity:{}".format(svc_2_cfmat[1:2, 1:2]/(svc_2_cfmat[1:2, 0:1] + svc_2_cfmat[1:2, 1:2])))

The Training score is: 0.6882042392246474
The Test score is: 0.6826347305389222
[[  70    0    0    0  181    9]
 [   3    0    0    0   20    0]
 [   4    0    0    0   92    1]
 [   6    0    0    0  226    1]
 [  28    0    0    0 2087    9]
 [  28    0    0    0  399    9]]


In [93]:
from sklearn.metrics import classification_report
print('Classification Report:\n {}'.format(classification_report(y4_test, y4_pred_)))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.50      0.27      0.35       260
         1.0       0.00      0.00      0.00        23
         2.0       0.00      0.00      0.00        97
         3.0       0.00      0.00      0.00       233
         4.0       0.69      0.98      0.81      2124
         5.0       0.31      0.02      0.04       436

   micro avg       0.68      0.68      0.68      3173
   macro avg       0.25      0.21      0.20      3173
weighted avg       0.55      0.68      0.58      3173



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##### Analysis
This performs similarly poor as the other classification model above. As proved in the binary models, it seems as if adding the nutrition facts back in does not help out all that much.

---